In [1]:
import h5py

In [3]:
h5_path = "/bd_byta6000i0/users/surgical_depth/SCARED_fixed/scared.hdf5"

In [4]:
h5file =  h5py.File(h5_path, "r")

In [5]:
h5file.keys()

<KeysViewHDF5 ['dataset1', 'dataset2', 'dataset3', 'dataset4', 'dataset5', 'dataset6', 'dataset7', 'dataset8', 'dataset9']>

In [6]:
h5file['dataset1'].keys()

<KeysViewHDF5 ['keyframe1', 'keyframe2', 'keyframe3']>

In [7]:
h5file['dataset1']['keyframe1'].keys()

<KeysViewHDF5 ['000001', '000002', '000003', '000004', '000005', '000006', '000007', '000008', '000009', '000010', '000011', '000012', '000013', '000014', '000015', '000016', '000017', '000018', '000019', '000020', '000021', '000022', '000023', '000024', '000025', '000026', '000027', '000028', '000029', '000030', '000031', '000032', '000033', '000034', '000035', '000036', '000037', '000038', '000039', '000040', '000041', '000042', '000043', '000044', '000045', '000046', '000047', '000048', '000049', '000050', '000051', '000052', '000053', '000054', '000055', '000056', '000057', '000058', '000059', '000060', '000061', '000062', '000063', '000064', '000065', '000066', '000067', '000068', '000069', '000070', '000071', '000072', '000073', '000074', '000075', '000076', '000077', '000078', '000079', '000080', '000081', '000082', '000083', '000084', '000085', '000086', '000087', '000088', '000089', '000090', '000091', '000092', '000093', '000094', '000095', '000096', '000097', '000098', '0000

In [8]:
h5file['dataset1']['keyframe1']['000001'].keys()

<KeysViewHDF5 ['gt', 'image']>

In [9]:
h5file['dataset1']['keyframe1']['000001']['gt']

<HDF5 dataset "gt": shape (1024, 1280, 1), type "<f4">

In [9]:
print(h5file['dataset1']['keyframe1']['000001']['image'].compression)

None


In [11]:
import h5py
import lance
import pyarrow as pa
import numpy as np
import os
import random

In [10]:
import h5py
import lance
import pyarrow as pa
import numpy as np
import os
import random

def convert_scared_hdf5_to_lance_hdd(h5_path, lance_path):
    # --- 1. 参数配置 (HDD 优化版) ---
    BATCH_SIZE = 50           # 控制内存：防止 OOM
    MAX_ROWS_PER_GROUP = 64   # 控制磁盘：HDD 需要更大的连续块 (约 560MB/组)
    H, W = 1024, 1280

    # 定义 Tensor 类型
    # Image: (H, W, 3) uint8
    image_tensor_type = pa.fixed_shape_tensor(pa.uint8(), (H, W, 3))
    # Depth: (H, W, 1) float32 (保留 Channel 维)
    depth_tensor_type = pa.fixed_shape_tensor(pa.float32(), (H, W, 1))

    # 定义 Schema
    schema = pa.schema([
        pa.field("dataset_id", pa.string()),
        pa.field("keyframe_id", pa.string()),
        pa.field("frame_name", pa.string()),
        pa.field("image", image_tensor_type),
        pa.field("depth", depth_tensor_type)
    ])

    # --- 2. 预扫描与打乱 (HDD 核心优化) ---
    print(f"正在扫描 HDF5 索引 (为了物理打乱数据)...")
    all_frames = []
    
    # 第一次只读 Key，不读数据，速度很快
    with h5py.File(h5_path, 'r') as f:
        for ds_id in f.keys():
            if not isinstance(f[ds_id], h5py.Group): continue
            for kf_id in f[ds_id].keys():
                if not isinstance(f[ds_id][kf_id], h5py.Group): continue
                for frame_name in f[ds_id][kf_id].keys():
                    # 检查必要字段是否存在
                    grp = f[ds_id][kf_id][frame_name]
                    if 'image' in grp and 'gt' in grp:
                        all_frames.append( (ds_id, kf_id, frame_name) )

    print(f"扫描完成，共找到 {len(all_frames)} 帧。正在打乱顺序...")
    random.shuffle(all_frames) # <--- 关键：物理乱序写入

    # --- 3. 辅助函数：构建 RecordBatch ---
    def _create_batch_from_buffer(buffer):
        if not buffer["dataset_id"]:
            return None

        # 构造基础列
        arrays = [
            pa.array(buffer["dataset_id"]),
            pa.array(buffer["keyframe_id"]),
            pa.array(buffer["frame_name"]),
        ]
        names = ["dataset_id", "keyframe_id", "frame_name"]

        # 构造 Image Tensor 列
        img_stack = np.stack(buffer["image"]) 
        img_arrow = pa.FixedShapeTensorArray.from_numpy_ndarray(img_stack)
        arrays.append(img_arrow)
        names.append("image")

        # 构造 Depth Tensor 列
        depth_stack = np.stack(buffer["depth"])
        depth_arrow = pa.FixedShapeTensorArray.from_numpy_ndarray(depth_stack)
        arrays.append(depth_arrow)
        names.append("depth")

        # 修复报错的关键：返回 RecordBatch 而不是 Table
        return pa.RecordBatch.from_arrays(arrays, names=names)

    # --- 4. 数据生成器 ---
    def batch_generator():
        buffer = {
            "dataset_id": [], "keyframe_id": [], "frame_name": [],
            "image": [], "depth": []
        }
        
        print(f"开始读取数据并写入...")
        # 保持文件打开状态进行迭代
        with h5py.File(h5_path, 'r') as f:
            for idx, (ds_id, kf_id, frame_name) in enumerate(all_frames):
                if idx % 100 == 0:
                    print(f"Processing {idx}/{len(all_frames)}...", end='\r')

                # 根据索引定位数据
                frame_group = f[ds_id][kf_id][frame_name]

                # 读取并处理
                img_array = frame_group['image'][:] 
                
                depth_array = frame_group['gt'][:]
                # 确保深度图是 (H, W, 1)
                if depth_array.ndim == 2:
                    depth_array = depth_array[..., np.newaxis]

                # 加入缓冲
                buffer["dataset_id"].append(ds_id)
                buffer["keyframe_id"].append(kf_id)
                buffer["frame_name"].append(frame_name)
                buffer["image"].append(img_array)
                buffer["depth"].append(depth_array.astype(np.float32))

                # 缓冲区满，Yield 一个 Batch
                if len(buffer["dataset_id"]) >= BATCH_SIZE:
                    batch = _create_batch_from_buffer(buffer)
                    if batch is not None:
                        yield batch
                    # 清空 Buffer
                    for key in buffer: buffer[key] = []
            
            # 处理剩余数据
            if len(buffer["dataset_id"]) > 0:
                batch = _create_batch_from_buffer(buffer)
                if batch is not None:
                    yield batch

    # --- 5. 执行写入 ---
    print(f"\n目标路径: {lance_path}")
    print(f"策略: Batch Size={BATCH_SIZE}, Group Size={MAX_ROWS_PER_GROUP}")
    
    lance.write_dataset(
        batch_generator(),
        lance_path,
        schema=schema,
        mode="overwrite",
        max_rows_per_group=MAX_ROWS_PER_GROUP # HDD 优化参数
    )
    print("\n写入全部完成！")

# --- 验证代码 ---
if __name__ == "__main__":
    # 配置路径
    input_h5 = "/bd_byta6000i0/users/surgical_depth/SCARED_fixed/scared.hdf5"
    output_lance = "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED_hdd.lance"
    
    if os.path.exists(input_h5):
        # 1. 运行转换
        convert_scared_hdf5_to_lance_hdd(input_h5, output_lance)
        
        # 2. 简单验证读取
        if os.path.exists(output_lance):
            print("-" * 30)
            print("验证读取性能...")
            ds = lance.dataset(output_lance)
            print(f"总行数: {ds.count_rows()}")
            
            # 随机取一行 (因为已经打乱了，take(0) 其实是随机的一张图)
            item = ds.take([0], columns=['image', 'depth']).to_pydict()
            
            img = item['image'][0]
            if hasattr(img, 'to_numpy'): img = img.to_numpy()
            
            depth = item['depth'][0]
            if hasattr(depth, 'to_numpy'): depth = depth.to_numpy()

            print(f"Image Info: Shape={img.shape}, Type={img.dtype}")
            print(f"Depth Info: Shape={depth.shape}, Type={depth.dtype}")
            print("数据完整性检查通过。")
    else:
        print(f"找不到输入文件: {input_h5}")

正在扫描 HDF5 索引 (为了物理打乱数据)...
扫描完成，共找到 28857 帧。正在打乱顺序...

目标路径: /bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED_hdd.lance
策略: Batch Size=50, Group Size=64
开始读取数据并写入...


[2025-12-11T09:50:29Z WARN  lance::dataset::write::insert] No existing dataset at /bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED_hdd.lance, it will be created


Processing 28800/28857...
写入全部完成！
------------------------------
验证读取性能...
总行数: 28857


AttributeError: 'list' object has no attribute 'shape'

In [13]:
output_lance = "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED_hdd.lance"


In [14]:
if os.path.exists(output_lance):
    print("-" * 30)
    print("验证读取性能...")
    ds = lance.dataset(output_lance)
    print(f"总行数: {ds.count_rows()}")
    
    # 随机取一行 (因为已经打乱了，take(0) 其实是随机的一张图)
    batch_table = ds.to_table(columns=["image", "depth"], limit=16)

------------------------------
验证读取性能...
总行数: 28857


In [15]:
batch_table

pyarrow.Table
image: extension<arrow.fixed_shape_tensor[value_type=uint8, shape=[1024,1280,3]]>
depth: extension<arrow.fixed_shape_tensor[value_type=float, shape=[1024,1280,1]]>
----
image: [[[102,60,97,104,61,...,7,28,35,7,28],[9,7,5,9,7,...,7,6,11,7,6],...,[51,29,25,54,32,...,9,8,14,9,8],[66,27,28,66,27,...,110,117,125,110,117]]]
depth: [[[0,0,0,0,0,...,0,0,0,0,0],[0,0,0,0,0,...,57.69668,0,0,0,0],...,[0,78.02751,0,79.12905,78.02449,...,0,0,0,0,0],[0,0,0,0,0,...,0,0,0,0,0]]]

In [28]:
tbl = ds.take([0], columns=["image", "depth"])

In [29]:
img_scalar = tbl["image"][0]

In [30]:
depth_scalar = tbl["depth"][0]

In [35]:
depth_scalar.to_numpy().shape

(1024, 1280, 1)

In [34]:
import torch
import lance.torch.data

# Load lance dataset into a PyTorch IterableDataset.
# with only columns "image" and "prompt".
dataset = lance.torch.data.LanceDataset(
    "/bd_byta6000i0/users/surgicaldinov2/kyyang/surgicaldinov3/data/SCARED_hdd.lance",
    columns=["image", 'depth'],
    batch_size=6,
    batch_readahead=8,  # Control multi-threading reads.
    shuffleeee=True
)

In [30]:
batch = next(iter(dataset))

In [31]:
images = batch["image"].reshape(-1, 1024, 1280, 3).permute(0, 3, 1, 2)

In [32]:
import matplotlib.pyplot as plt

In [ ]:
import torchvision.transforms.v2 as T
image_transform = T.Compose(
    [
        T.Resize(256),
        T.ToDtype(torch.float32, scale=True),
        T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

/bd_byta6000i0/users/surgicaldinov2/miniforge3/envs/da3/lib/python3.11/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [24]:
images_trans = image_transform(images)

In [28]:
images_trans.ndim

4